# Importing a library that is not in Colaboratory

To import a library that's not in Colaboratory by default, you can use `!pip install` or `!apt-get install`.

In [1]:
!pip install matplotlib-venn

In [5]:
!apt-get -qq install -y libfluidsynth1

E: Package 'libfluidsynth1' has no installation candidate


In [6]:

from fastapi import FastAPI, HTTPException, Query
from datetime import datetime, timedelta, UTC
from typing import Dict
import jwt
import base64
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
import uuid

app = FastAPI()

# Store keys and expiry metadata
keys: Dict[str, Dict] = {}
KEY_EXPIRY_HOURS = 1

def generate_rsa_key():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048
    )
    public_key = private_key.public_key()
    kid = str(uuid.uuid4())
    expiry = datetime.now(UTC) + timedelta(hours=KEY_EXPIRY_HOURS)

    private_pem = private_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
    public_pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    keys[kid] = {
        "private": private_pem.decode(),
        "public": public_pem.decode(),
        "expiry": expiry
    }
    return kid

def get_jwks():
    jwks_keys = []
    for kid, key_info in keys.items():
        if key_info["expiry"] > datetime.now(UTC):
            public_key = serialization.load_pem_public_key(key_info["public"].encode())
            public_numbers = public_key.public_numbers()
            n_bytes = public_numbers.n.to_bytes((public_numbers.n.bit_length() + 7) // 8, "big")

            jwks_keys.append({
                "kty": "RSA",
                "kid": kid,
                "alg": "RS256",
                "use": "sig",
                "n": base64.urlsafe_b64encode(n_bytes).rstrip(b"=").decode(),  # Correct base64 encoding
                "e": "AQAB"
            })

    return {"keys": jwks_keys}

@app.get("/jwks")
def jwks():
    return get_jwks()

@app.post("/auth")
def auth(expired: bool = Query(False)):
    if not keys:
        raise HTTPException(status_code=500, detail="No keys available")

    key_id = next(iter(keys.keys()))  # Always select the first key
    key_info = keys[key_id]

    expiry = key_info["expiry"] if not expired else datetime.now(UTC) - timedelta(hours=1)

    token = jwt.encode(
        {"sub": "fake_user", "exp": expiry},
        key_info["private"],
        algorithm="RS256",
        headers={"kid": key_id}
    )
    return {"token": token}

# Generate initial key
generate_rsa_key()

# Basic test cases
import unittest
from fastapi.testclient import TestClient

client = TestClient(app)

class TestJWKS(unittest.TestCase):
    def test_jwks(self):
        response = client.get("/jwks")
        self.assertEqual(response.status_code, 200)
        self.assertIn("keys", response.json())

    def test_auth(self):
        response = client.post("/auth")
        self.assertEqual(response.status_code, 200)
        self.assertIn("token", response.json())

    def test_auth_expired(self):
        response = client.post("/auth?expired=true")
        self.assertEqual(response.status_code, 200)
        self.assertIn("token", response.json())

if __name__ == "__main__":
    unittest.main()


ModuleNotFoundError: No module named 'fastapi'

# Install 7zip reader [libarchive](https://pypi.python.org/pypi/libarchive)

In [4]:
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 124926 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.6.0-1ubuntu1.3_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.6.0-1ubuntu1.3) ...
Setting up libarchive-dev:amd64 (3.6.0-1ubuntu1.3) ...
Processing triggers for man-db (2.10.2-1) ...
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 7.7 MB/s eta 0:00:00
  Created wheel for libarchive: filename=libarchive-0.4.7-py3-none-any.whl size=31631 sha256=4edbdefb04317830dbe41e63f2657de481d4acc49c7e0f17dcd52520f17263c8
  Stored in directory: /root/.cache/pip/wheels/32/98/bd/4893d6923dd027f455b250367d402bfd69a6f4416581df46db
Successfully built libarchive


# Install GraphViz & [PyDot](https://pypi.python.org/pypi/pydot)

In [3]:
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install pydot
import pydot

# Install [cartopy](http://scitools.org.uk/cartopy/docs/latest/)

In [2]:
!pip install cartopy
import cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 34.7 MB/s eta 0:00:00
